Maybe we define new structure which captures the input parameters specific to each problem in one place (`scale_factor`).
We need to define a range on `d` that is also function dependent (that we could adjust by hand). 

We need to generate three graphs: 
- Histogram of Number of `loc_min` points were found, (so outputs of the Optim routine) and what percentage of them is within a small distance of a critical point of the approximant. As a function of the degree `d` of the approximant.


In [ ]:
using Pkg
using Revise 
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging
using Optim
using CairoMakie
CairoMakie.activate!

Can a function export "Optional attributes" like just some constants or its optimal domain of definition? --> works. 

In [ ]:
params = load_function_params("HolderTable")
TR = test_input(HolderTable;
    dim=params.dim,
    center=params.center,
    GN=params.num_samples,
    sample_range=params.sample_range,
    tolerance=params.tolerance)

@polyvar(x[1:TR.dim]); # Define polynomial ring 

In [ ]:
d_min, d_max = 4, 30
TD = 0.5

In [ ]:
results = analyze_degrees(TR, x, d_min, d_max, step=1, tol_dist=TD)

In [ ]:
# new_results = analyze_degrees(TR, x, d_min, d_max, results, tol_dist=.5)

In [ ]:
fig_1 = plot_discrete_l2(results, d_min, d_max, 1)
# save("discrete_l2.pdf", fig_1)
display(fig_1)

In [ ]:
fig_2 = capture_histogram(results, d_min, d_max, 1, tol_dist=TD, show_legend = false)
# save("histogram.pdf", fig_2)
display(fig_2)

In [ ]:
fig_3 = plot_convergence_analysis(results, d_min, d_max, 1)
# save("convergence_analysis.pdf", fig_3)
display(fig_3)

In [ ]:
RT = results[18]
df_t = RT.df
df_m = RT.df_min;
# pol_cheb = Constructor(TR, 18, basis=:chebyshev)
sort!(df_t, :z)

Adding the mast here may not be that useful. We care of the minimal distance separating the optimized points from the critical points.

In [ ]:
inside_mask = points_in_hypercube(df_t, TR)
values_mask = points_in_range(df_t, TR, 18.)
df_minimizers = df_t[values_mask .& inside_mask, :] # has both `x` (raw) and `y` (optimized)

In [ ]:
CairoMakie.activate!
fig_1 = cairo_plot_polyapprox_levelset(pol_cheb, TR, df_t, df_m, show_captured=false)
# fig_1_p = cairo_plot_polyapprox_levelset(pol_cheb, TR, df_minimizers, df_m, show_captured=false)
# save("polyapprox_levelset_just_crit.pdf", fig_1)

In [ ]:
stats = analyze_converged_points(df_t, TR, results, d_min, d_max, 1)

In [ ]:
fig_5 = plot_distance_statistics(stats)
# save("distance_to_minimizer.pdf", fig_5)

In [ ]:
function plot_distance_statistics(stats::Dict{String,Any}; show_legend::Bool=true)
    fig = Figure(size=(600, 400))

    ax = Axis(fig[1, 1],
        xlabel="Degree")

    # Plot maximum and average distances
    degrees = stats["degrees"]
    scatterlines!(ax, degrees, stats["max_distances"],
        label="Maximum",
        color=:red)
    scatterlines!(ax, degrees, stats["avg_distances"],
        label="Average",
        color=:blue)

    if show_legend
        axislegend(ax)
    end

    return fig
end

In [ ]:
function analyze_converged_points(
    df_filtered::DataFrame,
    TR::test_input,
    results::Dict{Int,NamedTuple{(:df, :df_min, :convergence_stats, :discrete_l2),
        Tuple{DataFrame,DataFrame,NamedTuple,Float64}}},
    start_degree::Int,
    end_degree::Int,
    step::Int=1)

    degrees = start_degree:step:end_degree
    n_dims = count(col -> startswith(string(col), "x"), names(df_filtered))

    # Filter for converged points first
    df_converged = df_filtered[df_filtered.converged, :]

    # Filter for points where y is in domain and not NaN
    valid_points = trues(nrow(df_converged))
    for i in 1:nrow(df_converged)
        # Check if y coordinates are NaN
        y_coords = [df_converged[i, Symbol("y$j")] for j in 1:n_dims]
        if any(isnan.(y_coords))
            valid_points[i] = false
            continue
        end

        # Check if y coordinates are in domain
        for j in 1:n_dims
            if abs(df_converged[i, Symbol("y$j")] - TR.center[j]) > TR.sample_range
                valid_points[i] = false
                break
            end
        end
    end

    df_valid = df_converged[valid_points, :]
    n_valid_points = nrow(df_valid)

    # Initialize distance matrix
    point_distances = zeros(Float64, n_valid_points, length(degrees))

    # Calculate distances
    for (i, row) in enumerate(eachrow(df_valid))
        y_coords = [row[Symbol("y$j")] for j in 1:n_dims]

        for (d_idx, d) in enumerate(degrees)
            raw_points = results[d].df
            min_dist = Inf

            for raw_row in eachrow(raw_points)
                point = [raw_row[Symbol("x$j")] for j in 1:n_dims]
                dist = norm(y_coords - point)
                min_dist = min(min_dist, dist)
            end
            point_distances[i, d_idx] = min_dist
        end
    end

    # Calculate statistics
    stats = Dict{String,Any}()

    # Per-degree statistics
    stats["max_distances"] = [maximum(point_distances[:, i]) for i in 1:length(degrees)]
    stats["min_distances"] = [minimum(point_distances[:, i]) for i in 1:length(degrees)]
    stats["avg_distances"] = [mean(point_distances[:, i]) for i in 1:length(degrees)]

    # Overall statistics
    stats["overall_max"] = maximum(stats["max_distances"])
    stats["overall_min"] = minimum(stats["min_distances"])
    stats["overall_avg"] = mean(stats["avg_distances"])

    # Additional metadata
    stats["n_total_points"] = nrow(df_filtered)
    stats["n_converged"] = nrow(df_converged)
    stats["n_valid"] = n_valid_points
    stats["degrees"] = collect(degrees)

    return stats
end